
> 📌 **DOI:** [10.5281/zenodo.15636698](https://doi.org/10.5281/zenodo.15636698)  
This notebook is part of the FDI Observer Simulator project for secure estimation in wireless CPS.  
For citation and licensing, see the [Zenodo record](https://doi.org/10.5281/zenodo.15636698).


# Observer-Based Estimation under FDI and Packet Loss

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve_discrete_are
from gain_synthesis_lmi import synthesize_observer_gain
from sliding_mode_observer import SlidingModeObserver

In [ ]:
T = 100
n, m, p = 2, 1, 1
A = np.array([[1.0, 0.1], [0, 1.0]])
B = np.array([[0.0], [0.1]])
C = np.array([[1.0, 0.0]])
Q = np.eye(n)
R = np.eye(m)

In [ ]:
L = synthesize_observer_gain(A, C)
P = solve_discrete_are(A, B, Q, R)
K = np.linalg.inv(R + B.T @ P @ B) @ (B.T @ P @ A)

In [ ]:
x = np.zeros((n, T))
x_hat_luen = np.zeros((n, T))
x_hat_smo = np.zeros((n, T))
e_luen = np.zeros((n, T))
e_smo = np.zeros((n, T))
attack_strength = 0.2
z = attack_strength * np.random.randn(p, T)
w_std = 0.01
v_std = 0.01
drop_prob = 0.3
SMO = SlidingModeObserver(A, B, C, L, rho=0.05)
SMO.reset()

In [ ]:
for k in range(T - 1):
    u_k = -K @ x_hat_luen[:, k]
    w_k = w_std * np.random.randn(n)
    v_k = v_std * np.random.randn(p)
    x[:, k + 1] = A @ x[:, k] + B @ u_k + w_k
    y_k = C @ x[:, k] + z[:, k] + v_k

    if np.random.rand() < drop_prob:
        x_hat_luen[:, k + 1] = A @ x_hat_luen[:, k] + B @ u_k
        x_hat_smo[:, k + 1] = A @ x_hat_smo[:, k] + B @ u_k
    else:
        innovation = y_k - C @ x_hat_luen[:, k]
        x_hat_luen[:, k + 1] = A @ x_hat_luen[:, k] + B @ u_k + L @ innovation
        x_hat_smo[:, k + 1], _ = SMO.update(u_k, y_k)

    e_luen[:, k + 1] = x[:, k + 1] - x_hat_luen[:, k + 1]
    e_smo[:, k + 1] = x[:, k + 1] - x_hat_smo[:, k + 1]

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(np.linalg.norm(e_luen, axis=0), label='Luenberger')
plt.plot(np.linalg.norm(e_smo, axis=0), label='Sliding Mode')
plt.title('Estimation Error Norm Comparison (FDI + Packet Loss)')
plt.xlabel('Time Step')
plt.ylabel('||e_k||')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
attack_levels = np.linspace(0.0, 0.6, 6)
drop_probs = np.linspace(0.0, 0.6, 6)
mean_errors = np.zeros((len(attack_levels), len(drop_probs)))

for i, zeta in enumerate(attack_levels):
    for j, drop_prob in enumerate(drop_probs):
        x = np.zeros((n, T))
        x_hat = np.zeros((n, T))
        e = np.zeros((n, T))
        z = zeta * np.random.randn(p, T)

        for k in range(T - 1):
            u_k = -K @ x_hat[:, k]
            w_k = w_std * np.random.randn(n)
            v_k = v_std * np.random.randn(p)
            x[:, k + 1] = A @ x[:, k] + B @ u_k + w_k
            y_k = C @ x[:, k] + z[:, k] + v_k

            if np.random.rand() < drop_prob:
                x_hat[:, k + 1] = A @ x_hat[:, k] + B @ u_k
            else:
                innovation = y_k - C @ x_hat[:, k]
                x_hat[:, k + 1] = A @ x_hat[:, k] + B @ u_k + L @ innovation

            e[:, k + 1] = x[:, k + 1] - x_hat[:, k + 1]

        mean_errors[i, j] = np.mean(np.linalg.norm(e, axis=0))

In [ ]:
plt.figure(figsize=(8, 6))
plt.imshow(mean_errors, origin='lower', cmap='viridis',
           extent=(drop_probs[0], drop_probs[-1], attack_levels[0], attack_levels[-1]),
           aspect='auto')
plt.colorbar(label='Mean Estimation Error')
plt.xlabel('Packet Drop Probability (1 - $p_s$)')
plt.ylabel('Attack Strength ζ')
plt.title('Sensitivity Analysis: Error vs Attack Strength and Drop Rate')
plt.tight_layout()
plt.show()